# Test normalizzazione cella - Veicoli

In [9]:
import geolib.geohash
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import pandas as pd
from tqdm.auto import tqdm
from cri98tj.partitioners.Geohash_partitioner import Geohash_partitioner
from cri98tj.normalizers.FirstPoint_normalizer import FirstPoint_normalizer
from cri98tj.selectors.RandomInformationGain_selector import RandomInformationGain_selector
from cri98tj.distancers.Euclidean_distancer import Euclidean_distancer
from sklearn.model_selection import train_test_split
from cri98tj.distancers.Euclidean_distancer import euclideanBestFitting

In [10]:
df = pd.read_csv('../examples/Vehicles Dataset/data/vehicles_preapred.zip').sort_values(by=["tid", "t"])# precision=5, 50 movelet, DTW
df = df[["tid", "class", "c1", "c2", "t"]]
df["c1"] = df.c1/100000
df["c2"] = df.c2/100000
df.head()

,tid,class,c1,c2,t
0,30901,B,42.077160,4.738411,0
1,30901,B,42.077246,4.739088,30
2,30901,B,42.077259,4.739096,60
3,30901,B,42.077369,4.739158,90
4,30901,B,42.077635,4.739343,120


In [11]:
tid_train, tid_test, _, _ = train_test_split(df.groupby(by=["tid"]).max().reset_index()["tid"],
                                                        df.groupby(by=["tid"]).max().reset_index()["class"],
                                                        test_size=.3,
                                                        stratify=df.groupby(by=["tid"]).max().reset_index()["class"],
                                                        random_state=3)

spatioTemporalCols = ["c1", "c2", "t"]
n_jobs = 24
verbose = True
precision = 6

In [12]:
from cri98tj.distancers.InterpolatedRootDistance_distancer import InterpolatedRootDistanceBestFitting

normalizer = FirstPoint_normalizer(spatioTemporalColumns=spatioTemporalCols, fillna=None, verbose=verbose)
selector = RandomInformationGain_selector(top_k=20, bestFittingMeasure=InterpolatedRootDistanceBestFitting, movelets_per_class=300, trajectories_for_orderline=100, n_jobs=n_jobs, spatioTemporalColumns=spatioTemporalCols, normalizer=normalizer, verbose=verbose)

In [13]:
import random

partitioner = Geohash_partitioner(precision=precision, spatioTemporalColumns=spatioTemporalCols, verbose=verbose)
part = partitioner.fit_transform(df[df.tid.isin(tid_train)].values)
df_part = pd.DataFrame(part, columns=["tid", "class"]+spatioTemporalCols+["partId"])

n_per_class = int(100)
selected_tid = []
for classe in df_part["class"].unique():
    df_tmp = df_part[df_part["class"] == classe]
    selected_tid += random.sample(df_tmp.partId.unique().tolist(), k=min(len(df_tmp.partId.unique()), n_per_class))

df_shape = part[df_part.partId.isin(selected_tid)]

shapelets = normalizer.fit_transform(df_shape)

Encoding 125348 points with precision 6


  0%|          | 0/125348 [00:00<?, ?it/s]

Cutting sub-trajectories length at 8.0 over 18
Pivoting tables


  0%|          | 0/648 [00:00<?, ?it/s]

In [14]:
from geolib import geohash


def my_InterpolatedRootDistanceBestFitting(trajectory, movelet, spatioTemporalColumns):  # nan == end
    if len(trajectory) % len(spatioTemporalColumns) != 0:
        raise Exception(f"la lunghezza della traiettoria deve essere divisivile per {len(spatioTemporalColumns)}")
    if len(movelet) % len(spatioTemporalColumns) != 0:
        raise Exception(f"la lunghezza della traiettoria deve essere divisivile per {len(spatioTemporalColumns)}")

    offset_trajectory = int(len(trajectory) / len(spatioTemporalColumns))
    offset_movelet = int(len(movelet) / len(spatioTemporalColumns))

    len_mov = 0
    for el in movelet:
        if np.isnan(el) or len_mov >= offset_movelet:
            break
        len_mov += 1

    len_t = 0
    for el in trajectory:
        if np.isnan(el) or len_t >= offset_trajectory:
            break
        len_t += 1

    trajectory_dict = [None for x in spatioTemporalColumns]
    movelet_dict = [None for x in spatioTemporalColumns]

    for i, col in enumerate(spatioTemporalColumns):
        trajectory_dict[i] = trajectory[i * offset_trajectory:(i * offset_trajectory) + len_t]
        movelet_dict[i] = movelet[i * offset_movelet:(i * offset_movelet) + len_mov]

    bestScore = math.inf
    best_i = -1
    partitions = geohashPartition(trajectory_dict, precision)

    for i in range(len(partitions)): #mi assicuro di fare almeno 1 iterazione
        returned = trajectory_distance(partitions[i], movelet_dict)
        if returned is not None and returned < bestScore:
            bestScore = returned
            best_i = i

    return best_i, bestScore

def geohashPartition(trajectory_dict=[], precision=5):
    prec_index = 0
    prec_gh = geohash.encode(trajectory_dict[0][0], trajectory_dict[1][0], precision)
    prec_sw = geohash.bounds(prec_gh).sw
    partizioni = []
    for i, (c1, c2, t) in enumerate(zip(trajectory_dict[0], trajectory_dict[1], trajectory_dict[2])):
        if geohash.encode(c1, c2, precision) != prec_gh:
            dict_tmp = {}
            dict_tmp[0] = trajectory_dict[0][prec_index:i] - prec_sw.lat
            dict_tmp[1] = trajectory_dict[1][prec_index:i] - prec_sw.lon
            dict_tmp[2] = trajectory_dict[2][prec_index:i] - trajectory_dict[2][prec_index]
            partizioni.append(dict_tmp)

            prec_index = i
            prec_gh = geohash.encode(c1, c2, precision)
            prec_sw = geohash.bounds(prec_gh).sw
    dict_tmp = {}
    dict_tmp[0] = trajectory_dict[0][prec_index:]- prec_sw.lat
    dict_tmp[1] = trajectory_dict[1][prec_index:]- prec_sw.lon
    dict_tmp[2] = trajectory_dict[2][prec_index:]- trajectory_dict[2][prec_index]
    partizioni.append(dict_tmp)

    return partizioni

In [15]:
from cri98tj.normalizers.normalizer_utils import dataframe_pivot
from cri98tj.distancers.InterpolatedRootDistance_distancer import trajectory_distance
import math
import numpy as np

trajectories_df = df.copy()
trajectories_df["partId"] = trajectories_df.tid
df_pivot = dataframe_pivot(df=trajectories_df, maxLen=None, verbose=verbose, fillna_value=None, columns=spatioTemporalCols)

dist_matrix = np.zeros((len(df_pivot), len(shapelets)))



Pivoting tables


  0%|          | 0/178299 [00:00<?, ?it/s]

In [16]:
for i, traj in enumerate(tqdm(df_pivot.values[:, 1:])):
    for j, mov in enumerate(shapelets.values[:, 1:]):
        _, dist_matrix[i, j] = my_InterpolatedRootDistanceBestFitting(traj, mov, spatioTemporalCols)

  0%|          | 0/381 [00:00<?, ?it/s]

In [17]:
clf = RandomForestClassifier(max_depth=3, random_state=3, n_jobs=n_jobs, n_estimators=1000)

dist_np_df = pd.DataFrame(dist_matrix)

In [18]:
dist_matrix

array([[131.65701994, 240.96984519,   0.        , ...,   0.        ,
          0.        ,   0.        ],
       [ 75.00702483, 113.58897961,   0.        , ...,   0.        ,
          0.        ,   0.        ],
       [120.31483577, 203.39960502,   0.        , ...,   0.        ,
          0.        ,   0.        ],
       ...,
       [ 61.12943355,  59.03236511,   0.        , ...,   0.        ,
          0.        ,   0.        ],
       [ 76.92980557,  89.6107694 ,   0.        , ...,   0.        ,
          0.        ,   0.        ],
       [133.79884235,  61.99288739,   0.        , ...,   0.        ,
          0.        ,   0.        ]])

In [19]:

X = dist_matrix
y = df_pivot["class"].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, stratify=y, random_state=3)

In [20]:
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           B       0.65      0.45      0.54        33
           T       0.80      0.90      0.85        82

    accuracy                           0.77       115
   macro avg       0.73      0.68      0.69       115
weighted avg       0.76      0.77      0.76       115



Risultati: anche in questo caso, la normalizzazione basata sulla cella sembra essere peggiore di quella basata sul primo punto. (?? vs ??, nonostante siano state usate molte più shapelets)